In [7]:
import pandas as pd
import os
import json
dataset_list = ['banking', 'clinc', 'stackoverflow']
model_list = ['deepseek-chat', 'gpt-4o-mini']
for dataset_name in dataset_list:
    for ratio in [0.25, 0.5, 0.75]:
        candidate_list = json.load(open(f"../../data/{dataset_name}/{dataset_name}_{ratio}/train_taxnomy.json", 'r'))['root']
        df = pd.read_csv(f"../../data/{dataset_name}/origin_data/test.tsv", sep='\t')
        df['label'] = df['label'].apply(lambda x: x.replace('_', ' '))
        for model_name in model_list:
            for seed in range(1):
                ans = {'gold':[], 'pred':[], 'text':[], 'id': []}
                dir_name = f"outputs/{dataset_name}/{ratio}/{model_name}/seed{seed}"
                if os.path.exists(f"{dir_name}"):
                    for i in os.listdir(dir_name):
                        i = int(i[:-4])
                        content = open(f"{dir_name}/{i}.txt", 'r').read()
                        pred = 'Novel'
                        gold = 'Novel'
                        for can in candidate_list:
                            if can in content:
                                pred = can
                                break
                        ans['pred'].append(pred)
                        ans['text'].append(df['text'][i])
                        ans['id'].append(i)
                        if df['label'][i] in candidate_list:
                            gold = df['label'][i]
                        ans['gold'].append(gold)
                ans = pd.DataFrame(ans)
                if len(ans) > 0:
                    if not os.path.exists(f'results/{dataset_name}/{ratio}/{model_name}'):
                        os.makedirs(f'results/{dataset_name}/{ratio}/{model_name}')
                    ans.to_csv(f'results/{dataset_name}/{ratio}/{model_name}/{seed}.tsv', index=None, sep='\t')

In [8]:
from sklearn.metrics import classification_report
ans = {'method':[],'dataset':[],'ratio':[],'value':[],'metric':[]}
for dataset_name in dataset_list:
    for ratio in [0.25, 0.5, 0.75]:
        for model_name in model_list:
            for seed in range(1):
                file_path = f"results/{dataset_name}/{ratio}/{model_name}/{seed}.tsv"
                if os.path.exists(file_path):
                    df = pd.read_csv(file_path, sep='\t')
                    report = classification_report(df['gold'], df['pred'], output_dict=True)
                    metric_result = {}
                    metric_result['F1'] = report['macro avg']['f1-score']
                    metric_result['ACC'] = report['accuracy']
                    metric_result['N-F1'] = report['Novel']['f1-score']
                    metric_result['K-F1'] = sum([report[i]['f1-score'] for i in report if i not in ['Novel', 'accuracy', 'macro avg', 'weighted avg']]) / (df['gold'].nunique() - 1) if df['gold'].nunique() - 1 != 0 else 0
                    for k in metric_result:
                        ans['method'].append(model_name)
                        ans['dataset'].append(dataset_name)
                        ans['ratio'].append(ratio)
                        ans['value'].append(metric_result[k] * 100)
                        ans['metric'].append(k)

/data/chenxi/miniconda3/envs/acl25/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/data/chenxi/miniconda3/envs/acl25/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/data/chenxi/miniconda3/envs/acl25/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/

In [9]:
import torch
for dataset_name in dataset_list:
    for ratio in [0.25, 0.5, 0.75]:
        for model_name in model_list[:1]:
            for seed in range(1):
                file_path = f"results/{dataset_name}/{ratio}/{model_name}/{seed}.tsv"
                if os.path.exists(file_path):
                    df = pd.read_csv(file_path, sep='\t')
                idx_list = []
                dir_name = f"outputs/{dataset_name}/{ratio}/{model_name}/seed{seed}"
                if os.path.exists(f"{dir_name}"):
                    for i in os.listdir(dir_name):
                        i = int(i[:-4])
                        idx_list.append(i)

                preds = torch.load(f'../../outputs/{dataset_name}/{dataset_name}-{ratio}--1/Meta-Llama-3.1-8B-Instruct-1-2-2-20-binary_0.1_0.1_1.0-sft-qlora-dsz3-seed_0/ood_eval_scores/test_binary_preds.pt')
                golds = torch.load(f'../../outputs/{dataset_name}/{dataset_name}-{ratio}--1/Meta-Llama-3.1-8B-Instruct-1-2-2-20-binary_0.1_0.1_1.0-sft-qlora-dsz3-seed_0/ood_eval_scores/test_golds.pt')
                pred = preds[idx_list]
                gold = golds[idx_list]

                report = classification_report(gold, pred, output_dict=True)
                metric_result = {}
                metric_result['F1'] = report['macro avg']['f1-score']
                metric_result['ACC'] = report['accuracy']
                metric_result['N-F1'] = report['0']['f1-score']
                metric_result['K-F1'] = sum([report[i]['f1-score'] for i in report if i not in ['0', 'accuracy', 'macro avg', 'weighted avg']]) / (df['gold'].nunique() - 1) if df['gold'].nunique() - 1 != 0 else 0

                for k in metric_result:
                    ans['method'].append('\ourmodel')
                    ans['dataset'].append(dataset_name)
                    ans['ratio'].append(ratio)
                    ans['value'].append(metric_result[k] * 100)
                    ans['metric'].append(k)
ans = pd.DataFrame(ans)
ans.to_csv('../../table_results/csv/Close-LLM_result.csv', index=None)
ans

<>:29: SyntaxWarning: invalid escape sequence '\o'
<>:29: SyntaxWarning: invalid escape sequence '\o'
/tmp/ipykernel_1909880/2652289167.py:29: SyntaxWarning: invalid escape sequence '\o'
  ans['method'].append('\ourmodel')
/data/chenxi/miniconda3/envs/acl25/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/data/chenxi/miniconda3/envs/acl25/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/data/chenxi/miniconda3/envs/acl25/lib/python3.12/site-packages/sklearn/metrics/_classification

,method,dataset,ratio,value,metric
0,deepseek-chat,banking,0.25,53.295664,F1
1,deepseek-chat,banking,0.25,42.682927,ACC
2,deepseek-chat,banking,0.25,40.199336,N-F1
3,deepseek-chat,banking,0.25,53.984944,K-F1
4,gpt-4o-mini,banking,0.25,43.582147,F1
...,...,...,...,...,...
103,\ourmodel,stackoverflow,0.50,88.214507,K-F1
104,\ourmodel,stackoverflow,0.75,91.356494,F1
105,\ourmodel,stackoverflow,0.75,94.133697,ACC
106,\ourmodel,stackoverflow,0.75,95.905172,N-F1


In [10]:
ans['method'] = pd.Categorical(ans['method'], categories=['gpt-4o-mini', 'deepseek-chat', '\ourmodel'], ordered=True)
ans = ans.sort_values(['ratio', 'method', 'dataset', 'metric'])
ans = ans.pivot(index=['ratio', 'method'], columns=['dataset', 'metric'], values=['value'])
ans = ans.round(2)
ans

<>:1: SyntaxWarning: invalid escape sequence '\o'
<>:1: SyntaxWarning: invalid escape sequence '\o'
/tmp/ipykernel_1909880/615028707.py:1: SyntaxWarning: invalid escape sequence '\o'
  ans['method'] = pd.Categorical(ans['method'], categories=['gpt-4o-mini', 'deepseek-chat', '\ourmodel'], ordered=True)


value                                                   \
dataset             banking                       clinc                        
metric                  ACC     F1   K-F1   N-F1    ACC     F1   K-F1   N-F1   
ratio method                                                                   
0.25  gpt-4o-mini     31.45  43.58  47.25  24.88  30.24  42.37  48.75  20.34   
      deepseek-chat   42.68  53.30  53.98  40.20  42.67  52.14  52.46  40.11   
      \ourmodel       91.77  78.23  77.34  95.08  95.20  84.35  79.44  97.14   
0.50  gpt-4o-mini     41.26  53.59  59.55   5.77  43.90  52.74  60.76   9.52   
      deepseek-chat   47.56  58.61  59.42  27.84  59.20  67.63  67.88  49.01   
      \ourmodel       91.31  80.27  77.79  94.36  94.80  91.85  86.89  96.53   
0.75  gpt-4o-mini     50.00  57.21  62.72   0.00  56.59  58.92  65.77   0.00   
      deepseek-chat   58.08  66.61  67.58  11.40  61.07  74.63  75.30   0.00   
      \ourmodel       92.53  91.12  91.07  93.89  95.47  91.42  83.21  96.41   

                                                        
dataset             stackoverflow                       
metric                        ACC     F1   K-F1   N-F1  
ratio method                                            
0.25  gpt-4o-mini           70.49  67.76  65.82  77.44  
      deepseek-chat         71.08  71.00  69.85  76.74  
      \ourmodel             97.04  90.70  89.18  98.30  
0.50  gpt-4o-mini           78.26  78.22  78.41  76.34  
      deepseek-chat         77.87  80.61  81.12  75.44  
      \ourmodel             93.63  88.90  88.21  95.72  
0.75  gpt-4o-mini           82.13  85.40  87.00  61.54  
      deepseek-chat         83.90  87.07  88.08  71.83  
      \ourmodel             94.13  91.36  91.05  95.91

In [11]:
latex_text = ans.to_latex()
latex_text = latex_text.replace('gpt-4o-mini', 'GPT-4o-mini').replace('deepseek-chat', 'DeepSeek-V3').replace(' ', '').replace('0000', '').replace('\cline{1-14}', '\midrule')
print(latex_text)

\begin{tabular}{llrrrrrrrrrrrr}
\toprule
&&\multicolumn{12}{r}{value}\\
&dataset&\multicolumn{4}{r}{banking}&\multicolumn{4}{r}{clinc}&\multicolumn{4}{r}{stackoverflow}\\
&metric&ACC&F1&K-F1&N-F1&ACC&F1&K-F1&N-F1&ACC&F1&K-F1&N-F1\\
ratio&method&&&&&&&&&&&&\\
\midrule
\multirow[t]{3}{*}{0.25}&GPT-4o-mini&31.45&43.58&47.25&24.88&30.24&42.37&48.75&20.34&70.49&67.76&65.82&77.44\\
&DeepSeek-V3&42.68&53.30&53.98&40.20&42.67&52.14&52.46&40.11&71.08&71.00&69.85&76.74\\
&\ourmodel&91.77&78.23&77.34&95.08&95.20&84.35&79.44&97.14&97.04&90.70&89.18&98.30\\
\midrule
\multirow[t]{3}{*}{0.50}&GPT-4o-mini&41.26&53.59&59.55&5.77&43.90&52.74&60.76&9.52&78.26&78.22&78.41&76.34\\
&DeepSeek-V3&47.56&58.61&59.42&27.84&59.20&67.63&67.88&49.01&77.87&80.61&81.12&75.44\\
&\ourmodel&91.31&80.27&77.79&94.36&94.80&91.85&86.89&96.53&93.63&88.90&88.21&95.72\\
\midrule
\multirow[t]{3}{*}{0.75}&GPT-4o-mini&50.00&57.21&62.72&0.00&56.59&58.92&65.77&0.00&82.13&85.40&87.00&61.54\\
&DeepSeek-V3&58.08&66.61&67.58&11.40&61.0

<>:2: SyntaxWarning: invalid escape sequence '\c'
<>:2: SyntaxWarning: invalid escape sequence '\m'
<>:2: SyntaxWarning: invalid escape sequence '\c'
<>:2: SyntaxWarning: invalid escape sequence '\m'
/tmp/ipykernel_1909880/2563063742.py:2: SyntaxWarning: invalid escape sequence '\c'
  latex_text = latex_text.replace('gpt-4o-mini', 'GPT-4o-mini').replace('deepseek-chat', 'DeepSeek-V3').replace(' ', '').replace('0000', '').replace('\cline{1-14}', '\midrule')
/tmp/ipykernel_1909880/2563063742.py:2: SyntaxWarning: invalid escape sequence '\m'
  latex_text = latex_text.replace('gpt-4o-mini', 'GPT-4o-mini').replace('deepseek-chat', 'DeepSeek-V3').replace(' ', '').replace('0000', '').replace('\cline{1-14}', '\midrule')
